### Imports

In [ ]:
%matplotlib widget
import sys
import os
sys.path.append(os.getcwd()+'/../../..')
import logging
logging.basicConfig(level=logging.INFO)
import re

from linetimer import CodeTimer
import nibabel
from nibabel import affines, nifti1
import numpy as np
import pyvista
pyvista.set_jupyter_backend('trame')
import trimesh

from BabelBrain.GPUFunctions.GPUVoxelize import Voxelize
from BabelBrain.BabelDatasetPreps import FixMesh

### GPU Initialization

In [ ]:
# GPUBackend='CUDA'
# GPUBackend='OpenCL'
GPUBackend='MLX'
gpu_name = 'Apple M3 Max'

Voxelize.InitVoxelize(gpu_name,GPUBackend=GPUBackend)

### Load Input Data

In [ ]:
T1W_fname = '/Users/spichardo/Documents/TempForSim/SDR_0p55/T1W.nii.gz'
stl_fname = '/Users/spichardo/Documents/TempForSim/SDR_0p55/m2m_SDR_0p55/skin.stl'
base_name = os.path.splitext(stl_fname)[0]

# Load data
nifti = nibabel.load(T1W_fname)
data = nifti.get_fdata().astype(np.uint8)
mesh = trimesh.load_mesh(stl_fname)

# Fix mesh if needed
if mesh.body_count != 1:
    print("Mesh needs fixing")
    base_name += '_fixed'
    fname = base_name + '.stl'
    if os.path.exists(fname):
        print("Reloading fixed mesh")
        mesh = trimesh.load_mesh(fname)
    else:
        print("Fixing mesh file")
        mesh = FixMesh(mesh)    
        mesh.export(base_name + '.stl')

### Determine Output Data Resolution

In [ ]:
# Set desired spatial step
resolution_level = {
    '0': 0.919,    # 200 kHz,   6 PPW
    '1': 0.613,    # 200 kHz,   9 PPW
    '2': 0.459,    # 200 kHz,  12 PPW
    '3': 0.306,    # 600 kHz,   6 PPW
    '4': 0.204,    # 600 kHz,   9 PPW
    '5': 0.153,    # 600 kHz,  12 PPW
    '6': 0.184,    # 1000 kHz,  6 PPW
    '7': 0.123,    # 1000 kHz,  9 PPW
    '8': 0.092,    # 1000 kHz, 12 PPW
}
spatial_step = np.full(3,resolution_level['5'])
# spatial_step = np.asarray(nifti.header.get_zooms())/2
spatial_step_text = re.sub("\.","_",str(spatial_step[0]))

# Calculate new affine
zooms = np.asarray(nifti.header.get_zooms())
print(f"Original zooms: {zooms}")
print(f"New zooms: {spatial_step}")
new_x = int(nifti.shape[0]/(spatial_step[0]/zooms[0]))
new_y = int(nifti.shape[1]/(spatial_step[1]/zooms[1]))
new_z = int(nifti.shape[2]/(spatial_step[2]/zooms[2]))
affine_upscaled = affines.rescale_affine(nifti.affine.copy(),
                                         nifti.shape,
                                         spatial_step,
                                         (new_x,new_y,new_z))

### Run Voxelization Step

In [ ]:
with CodeTimer("GPU Voxelization", unit="s"):
    points_voxelization_gpu=Voxelize.Voxelize(mesh,targetResolution=spatial_step[0], GPUBackend=GPUBackend)
    
with CodeTimer("CPU Voxelization", unit="s"):
    print(f"Starting Voxelization")
    voxelization_truth_fname = base_name + f"_voxelization_CPU_spatial_step_{spatial_step_text}.npy"
    if os.path.exists(voxelization_truth_fname):
        print("Reloading CPU file")
        points_voxelization_cpu = np.load(voxelization_truth_fname)
    else:
        print('Generating CPU file')
        points_voxelization_cpu = mesh.voxelized(spatial_step[0],max_iter=30).fill().points
        print('Saving CPU file')
        np.save(voxelization_truth_fname,points_voxelization_cpu)

# Ensure arrays are same size and dtype
gpu_vert_num = len(points_voxelization_gpu[:,0])
cpu_vert_num = len(points_voxelization_cpu[:,0])
if gpu_vert_num != cpu_vert_num:
    print(f"Array sizes don't match: {gpu_vert_num} vs {cpu_vert_num}")
if gpu_vert_num == 0:
    print("Arrays are empty")

In [ ]:
7.7/176.2

### Plot Data

In [ ]:
logging.basicConfig(level=logging.ERROR)

In [ ]:

mesh_step = points_voxelization_cpu.shape[0]//100000
mesh_cpu =  pyvista.PolyData(points_voxelization_cpu[::mesh_step,:])
mesh_gpu =  pyvista.PolyData(points_voxelization_gpu[::mesh_step,:])
plotter = pyvista.Plotter(shape=(1,2))
plotter.subplot(0,0)
plotter.add_mesh(pyvista.wrap(mesh),opacity=0.5)
plotter.add_mesh(mesh_cpu,color='blue',opacity=0.1)
plotter.add_text('CPU Voxelization', position='upper_edge', font_size=18)
plotter.subplot(0,1)
plotter.add_mesh(pyvista.wrap(mesh),opacity=0.5)
plotter.add_mesh(mesh_gpu,color='blue',opacity=0.1)
plotter.add_text('GPU Voxelization', position='upper_edge', font_size=18)
plotter.show()


### Check Output Data Matches Truth Data

In [ ]:
# Convert voxels back to 3D indices
inds_voxelization_gpu = np.hstack((points_voxelization_gpu,np.ones((points_voxelization_gpu.shape[0],1),dtype=points_voxelization_gpu.dtype))).T
inds_voxelization_cpu = np.hstack((points_voxelization_cpu,np.ones((points_voxelization_cpu.shape[0],1),dtype=points_voxelization_cpu.dtype))).T
ijk_gpu_tmp = np.round(np.linalg.inv(affine_upscaled).dot(inds_voxelization_gpu)).T
ijk_cpu_tmp = np.round(np.linalg.inv(affine_upscaled).dot(inds_voxelization_cpu)).T
ijk_gpu = np.ascontiguousarray(ijk_gpu_tmp[:,:3])
ijk_cpu = np.ascontiguousarray(ijk_cpu_tmp[:,:3])

# Remove duplicates
ijk_gpu_unique = np.unique(ijk_gpu, axis=0,)
ijk_cpu_unique = np.unique(ijk_cpu, axis=0)
print(f"Number of CPU indexes: {ijk_cpu_unique.shape[0]}")
print(f"Number of GPU indexes: {ijk_gpu_unique.shape[0]}")

# Count number of matches
set1 = set(map(tuple, ijk_gpu_unique))
set2 = set(map(tuple, ijk_cpu_unique))
common_coordinates = set1.intersection(set2)
match_count = len(common_coordinates)

# Calculate DICE coefficient
dice_coeff = 2 * match_count / (ijk_gpu_unique.shape[0] + ijk_cpu_unique.shape[0])
print(f"Dice coefficient: {dice_coeff}")